### **Adaptadores en PyTorch**

En PyTorch, afinar un modelo preentrenado puede hacerse modificando únicamente la capa final o ajustando todas las capas: el primero suele resultar en un rendimiento limitado; el segundo, en un gran consumo de recursos. 

Los **adaptadores** (adapters) ofrecen una alternativa más ligera: se insertan pequeños módulos entrenables dentro de la arquitectura original y solo éstos se actualizan durante el fine-tuning, manteniendo intactos los pesos preentrenados. De este modo, se ahorra tiempo y memoria, se reduce el riesgo de sobreajuste y se facilita la reutilización del mismo backbone para múltiples tareas. No obstante, al trabajar con adaptadores, es posible que la precisión final quede algo por debajo de la de un ajuste completo, especialmente en tareas que requieren modificaciones profundas del modelo. 

En el cuaderno práctico que sigue, aplicaremos un adaptador a un transformer entrenado sobre AG News para transferirlo al conjunto IMDB y compararemos su desempeño frente a un fine-tuning parcial (solo capa final) y uno completo.



### Configuración

#### Instalar las librerías necesarias

Para este cuaderno necesitaremos las siguientes librerías, que **no** vienen preinstaladas.**Debes ejecutar la siguiente celda** para instalarlas:

In [ ]:
#!pip install --upgrade portalocker==2.8.2 torchtext==0.17.0 torchdata==0.7.1 pandas==2.2.2 matplotlib==3.9.0 scikit-learn==1.5.0 torch==2.2.0 numpy==1.26.4

#### Importar las librerías requeridas

El siguiente bloque importa todas las librerías necesarias:


In [ ]:
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
from itertools import accumulate
import matplotlib.pyplot as plt
import math

import torch
torch.set_num_threads(1)
from torch import nn
import os


from torch.utils.data import DataLoader

from torchtext.datasets import AG_NEWS
from IPython.display import Markdown as md

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, GloVe, Vectors
from torchtext.datasets import IMDB
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

import pickle

from urllib.request import urlopen
import io

import tarfile
import tempfile

from torch.nn.utils.rnn import pad_sequence

# También puedes usar esta sección para suprimir las advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

#### **Funciones auxiliares**




In [ ]:
def plot(COST,ACC):
    """
    Grafica la evolución de la pérdida total y la precisión por época.

    Parámetros:
        COST (list): Lista con el valor de la pérdida total en cada época.
        ACC (list): Lista con la precisión en cada época.
    """

    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.plot(COST, color=color)
    ax1.set_xlabel('epoca', color=color)
    ax1.set_ylabel('Perdida total', color=color)
    ax1.tick_params(axis='y', color=color)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('accuracy', color=color)  
    ax2.plot(ACC, color=color)
    ax2.tick_params(axis='y', color=color)
    fig.tight_layout() 

    plt.show()

In [ ]:
def save_list_to_file(lst, filename):
    """
    Guarda una lista en un archivo usando serialización con pickle.

    Parámetros:
        lst (list): La lista que se guardará.
        filename (str): El nombre del archivo donde se guardará la lista.

    Retorna:
        None
    """
    with open(filename, 'wb') as file:
        pickle.dump(lst, file)

def load_list_from_file(filename):
    """
    Carga una lista desde un archivo usando deserialización con pickle.

    Parámetros:
        filename (str): El nombre del archivo desde el cual se cargará la lista.

    Retorna:
        list: La lista cargada.
    """
    with open(filename, 'rb') as file:
        loaded_list = pickle.load(file)
    return loaded_list

#### **Codificaciones posicionales**

Las codificaciones posicionales desempeñan un papel fundamental en los transformers y en diversos modelos de secuencia a secuencia, ya que ayudan a transmitir información crítica sobre las posiciones o el orden de los elementos dentro de una secuencia.

A pesar de sus significados distintos, cabe destacar que los *embeddings* de estas oraciones permanecen idénticos en ausencia de codificaciones posicionales. La siguiente clase define las codificaciones posicionales heredando de la clase `Module` de PyTorch.


In [ ]:
class PositionalEncoding(nn.Module):
    """
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

#### **Importar el conjunto de datos IMDB**

El siguiente código carga el conjunto de datos IMDB:


In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/35t-FeC-2uN1ozOwPs7wFg.gz')
tar = tarfile.open(fileobj=io.BytesIO(urlopened.read()))
tempdir = tempfile.TemporaryDirectory()
tar.extractall(tempdir.name)
tar.close()

#### **Descripción general del conjunto de datos IMDB**

El **conjunto de datos IMDB** contiene reseñas de películas extraídas de la Internet Movie Database (IMDB) y se utiliza habitualmente para tareas de clasificación de sentimientos binarios. Es un conjunto de datos de referencia para entrenar y evaluar modelos de procesamiento de lenguaje natural (NLP), en particular en análisis de sentimientos.

**Composición del conjunto de datos**

* **Reseñas**: El conjunto de datos consta de 50 000 reseñas de películas, divididas equitativamente en 25 000 muestras de entrenamiento y 25 000 de prueba.
* **Etiquetas de sentimiento**: Cada reseña está etiquetada como positiva o negativa, indicando el sentimiento expresado. El conjunto está balanceado, con igual número de reseñas positivas y negativas en ambos subconjuntos.
* **Contenido de texto**: Las reseñas se presentan en texto plano y han sido preprocesadas hasta cierto punto (por ejemplo, se han eliminado etiquetas HTML), aunque conservan puntuación y mayúsculas originales.
* **Uso**: Se emplea comúnmente para entrenar modelos de clasificación binaria de sentimientos, donde el objetivo es predecir si una reseña dada es positiva o negativa en función de su contenido textual.

**Aplicaciones**

* **Análisis de sentimientos**: Principal uso del conjunto IMDB como benchmark para distintos algoritmos de clasificación de texto.
* **Procesamiento de lenguaje natural**: Ampliamente utilizado en investigación y aplicaciones de NLP para comprobar la eficacia de modelos y enfoques en la comprensión del lenguaje humano.

#### **Desafíos**

* **Tamaño del conjunto**: Al ser relativamente pequeño, resulta difícil entrenar un modelo eficaz desde cero sin recurrir a técnicas de transferencia o regularización.



In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, root_dir, train=True):
        """
        root_dir: El directorio base del conjunto de datos IMDB.
        train: Indicado booleano que indica si se debe usar datos de entrenamiento o de prueba.
        """
        self.root_dir = os.path.join(root_dir, "train" if train else "test")
        self.neg_files = [os.path.join(self.root_dir, "neg", f) for f in os.listdir(os.path.join(self.root_dir, "neg")) if f.endswith('.txt')]
        self.pos_files = [os.path.join(self.root_dir, "pos", f) for f in os.listdir(os.path.join(self.root_dir, "pos")) if f.endswith('.txt')]
        self.files = self.neg_files + self.pos_files
        self.labels = [0] * len(self.neg_files) + [1] * len(self.pos_files)
        self.pos_inx=len(self.pos_files)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        label = self.labels[idx]
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return label, content

El siguiente código utiliza la clase `IMDBDataset` definida anteriormente para crear iteradores de los conjuntos de datos de entrenamiento y prueba. Luego, muestra 20 ejemplos del conjunto de entrenamiento:


In [ ]:
root_dir = tempdir.name + '/' + 'imdb_dataset'
train_iter = IMDBDataset(root_dir=root_dir, train=True)  # Para datos de entrenamiento
test_iter = IMDBDataset(root_dir=root_dir, train=False)  # Para datos de prueba

start=train_iter.pos_inx
for i in range(-10,10):
    print(train_iter[start+i])

El siguiente fragmento define el mapeo de etiquetas numéricas a reseñas negativas y positivas:


In [ ]:
imdb_label = {0: " negative review", 1: "positive review"}
imdb_label[1]

Por último, este código verifica que en el conjunto de entrenamiento existan exactamente dos clases:


In [ ]:
num_class = len(set([label for (label, text) in train_iter]))
num_class

El siguiente código carga un tokenizador básico en inglés y define una función llamada `yield_tokens` que utiliza el tokenizador para descomponer datos de texto proporcionados por un iterador en tokens:


In [ ]:
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    """Devuelve tokens para cada muestra de datos."""
    for _, text in data_iter:
        yield tokenizer(text)

El siguiente código carga un modelo de embeddings de palabras preentrenado llamado GloVe en una variable llamada `glove_embedding`:


In [ ]:
# Nota: los embeddings de GloVe normalmente se descargan usando:
# glove_embedding = GloVe(name="6B", dim=100)
# Sin embargo, el servidor de GloVe frecuentemente está inactivo.
# El código siguiente ofrece una solución alternativa.

class GloVe_override(Vectors):
    url = {
        "6B": "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/tQdezXocAJMBMPfUJx_iUg/glove-6B.zip",
    }

    def __init__(self, name="6B", dim=100, **kwargs) -> None:
        url = self.url[name]
        name = "glove.{}.{}d.txt".format(name, str(dim))
        #name = "glove.{}/glove.{}.{}d.txt".format(name, name, str(dim))
        super(GloVe_override, self).__init__(name, url=url, **kwargs)

class GloVe_override2(Vectors):
    url = {
        "6B": "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/tQdezXocAJMBMPfUJx_iUg/glove-6B.zip",
    }

    def __init__(self, name="6B", dim=100, **kwargs) -> None:
        url = self.url[name]
        #name = "glove.{}.{}d.txt".format(name, str(dim))
        name = "glove.{}/glove.{}.{}d.txt".format(name, name, str(dim))
        super(GloVe_override2, self).__init__(name, url=url, **kwargs)

try:
    glove_embedding = GloVe_override(name="6B", dim=100)
except:
    try:
        glove_embedding = GloVe_override2(name="6B", dim=100)
    except:
        glove_embedding = GloVe(name="6B", dim=100)

El siguiente código construye un objeto vocabulario a partir de un modelo de embeddings de palabras GloVe preentrenado y establece el índice predeterminado en el token `<unk>`:


In [ ]:
from torchtext.vocab import GloVe,vocab
# Construye el vocabulario a partir de glove_embedding.stoi
vocab = vocab(glove_embedding .stoi, 0,specials=('<unk>', '<pad>'))
vocab.set_default_index(vocab["<unk>"])

Contemos el número de palabras en el vocabulario:


In [ ]:
vocab_size=len(vocab)
vocab_size

Probemos la función `vocab`:


In [ ]:
vocab(['he'])

#### **División del conjunto de datos**

Se toman los iteradores de entrenamiento y de prueba y se convierten en conjuntos de datos del tipo "map".  A continuación, se aplica una partición aleatoria sobre el conjunto de entrenamiento original, reservando el 95 % de las muestras para entrenar y el 5 % restante para validar. 

De este modo obtenemos dos subconjuntos, uno de entrenamiento y otro de validación que se usan para ajustar y supervisar el modelo de clasificación de texto en IMDB. Finalmente, el rendimiento definitivo se mide sobre el conjunto de prueba independiente.



In [ ]:
# Convierte los iteradores de entrenamiento y prueba en conjuntos de datos de estilo map.
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

# Determina el número de muestras para entrenamiento y validación (5 % para validación).
num_train = int(len(train_dataset) * 0.95)

# Divide aleatoriamente el conjunto de entrenamiento en entrenamiento y validación.
# El conjunto de entrenamiento tendrá el 95 % de las muestras y el de validación el 5 % restante.
split_train_, split_valid_ = random_split(
    train_dataset,
    [num_train, len(train_dataset) - num_train]
)

Para simular el proceso como si se tuviera GPU, reducimos aún más el tamaño del conjunto de entrenamiento. Si deseas usar el conjunto IMDB completo, comenta o elimina las dos líneas siguientes:


In [ ]:
num_train = int(len(train_dataset) * 0.05)
split_train_, _ = random_split(split_train_, [num_train, len(split_train_) - num_train])

El siguiente código verifica si hay una GPU compatible con CUDA disponible usando PyTorch. Si existe, asigna `device = "cuda"`, de lo contrario `device = "cpu"`:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

#### **Cargador de datos**

El siguiente código prepara el pipeline de procesamiento de texto con el tokenizador y el vocabulario. 

La función `text_pipeline` primero tokeniza el texto de entrada y luego aplica `vocab` para obtener los índices de los tokens:

In [ ]:
def text_pipeline(x):
    return vocab(tokenizer(x))

En PyTorch, la función **`collate_fn`** se utiliza junto con los data loaders para personalizar la forma en que se crean los lotes a partir de muestras individuales. El código proporcionado define una función `collate_batch` en PyTorch, que se emplea con los data loaders para ajustar la creación de lotes a partir de muestras individuales. Esta función procesa un lote de datos, incluyendo etiquetas y secuencias de texto. 

Aplica la función `text_pipeline` para preprocesar el texto. Los datos resultantes se convierten en tensores de PyTorch y se devuelven como una tupla que contiene el tensor de etiquetas, el tensor de texto y un tensor de offsets que representa las posiciones iniciales de cada secuencia de texto dentro del tensor combinado.

Además, la función se asegura de que los tensores generados se muevan al dispositivo especificado (por ejemplo, GPU) para un cálculo más eficiente.


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list = [], []
    for _label, _text in batch:

        label_list.append(_label)
        text_list.append(torch.tensor(text_pipeline(_text), dtype=torch.int64))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True)

    return label_list.to(device), text_list.to(device)

Puedes convertir estos conjuntos de datos en data loaders aplicando `collate_fn`:

In [ ]:
BATCH_SIZE = 32

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)


Comprobemos qué generan estos data loaders:


In [ ]:
label,seqence=next(iter(valid_dataloader))
label,seqence

### **Red neuronal**

Este código define una clase llamada `Net` que representa un clasificador de texto basado en un `TransformerEncoder` de PyTorch.

El constructor recibe los siguientes argumentos:

* `num_class`: Número de clases a clasificar.
* `vocab_size`: Tamaño del vocabulario.
* `freeze`: Indica si se debe congelar la capa de embedding.
* `nhead`: Número de cabezas en el codificador Transformer.
* `dim_feedforward`: Dimensión de la capa feedforward en el codificador Transformer.
* `num_layers`: Número de capas del codificador Transformer.
* `dropout`: Tasa de dropout.
* `activation`: Función de activación que se usará en el codificador Transformer.
* `classifier_dropout`: Tasa de dropout para el clasificador.

**Atributos:**

* `emb`: Capa de embedding que mapea cada palabra del vocabulario a un vector denso.
* `pos_encoder`: Capa de codificación posicional que añade información de posición a los vectores de palabra.
* `transformer_encoder`: Codificador Transformer que procesa la secuencia de vectores de palabra y extrae características de alto nivel.
* `classifier`: Capa lineal que mapea la salida del codificador Transformer al número deseado de clases.

In [ ]:
class Net(nn.Module):
    """
    Clasificador de texto basado en un TransformerEncoder de PyTorch.
    """
    def __init__(

        self,
        num_class,vocab_size,
        freeze=True,
        nhead=2,
        dim_feedforward=128,
        num_layers=2,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1):

        super().__init__()

        #self.emb = embedding=nn.Embedding.from_pretrained(glove_embedding.vectors,freeze=freeze)
        self.emb = nn.Embedding.from_pretrained(glove_embedding.vectors,freeze=freeze)
        embedding_dim = self.emb.embedding_dim


        self.pos_encoder = PositionalEncoding(
            d_model=embedding_dim,
            dropout=dropout,
            vocab_size=vocab_size,
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.classifier = nn.Linear(embedding_dim, num_class)
        self.d_model = embedding_dim

    def forward(self, x):
        x = self.emb(x) * math.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.classifier(x)

        return x

El modelo puede entrenarse luego con datos etiquetados del conjunto IMDB, que tiene dos clases.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo = Net(num_class=2,vocab_size=vocab_size).to(device)
modelo

La siguiente función **`predict`** recibe un texto, un pipeline de procesamiento de texto y el modelo, y devuelve la etiqueta predicha usando el modelo preentrenado para clasificación en IMDB:


In [ ]:
def predict(text, text_pipeline, modelo):
    with torch.no_grad():
        text = torch.unsqueeze(torch.tensor(text_pipeline(text)),0).to(device)
        modelo.to(device)
        output = modelo(text)
        return imdb_label[output.argmax(1).item()]

In [ ]:
predict("I like sports and stuff", text_pipeline, modelo)

Para evaluar la precisión del modelo en un conjunto de datos, definimos dos funciones casi idénticas. Una muestra una barra de progreso con `tqdm` y la otra no:


In [ ]:
def evaluate(dataloader, model_eval):
    model_eval.eval()
    total_acc, total_count= 0, 0

    with torch.no_grad():
        for label, text in tqdm(dataloader):
            label, text = label.to(device), text.to(device)
            output = model_eval(text)
            predicted = torch.max(output.data, 1)[1]
            total_acc += (predicted == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

El siguiente código evalúa el rendimiento del modelo (puede tardar unos 4 minutos en CPU). **Para mayor eficiencia, no ejecutaremos esta celda ahora**, pero puedes descomentar la celda si se desea comprobar que el modelo sin entrenar no rinde mejor que el azar:



In [ ]:
def evaluate_no_tqdm(dataloader, model_eval):
    model_eval.eval()
    total_acc, total_count= 0, 0

    with torch.no_grad():
        for label, text in dataloader:
            label, text = label.to(device), text.to(device)
            output = model_eval(text)
            predicted = torch.max(output.data, 1)[1]
            total_acc += (predicted == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [ ]:
#evaluate(test_dataloader, modelo)

Ten en cuenta que el rendimiento actual del modelo no es mejor que el promedio. Este resultado es esperado, considerando que el modelo aún no ha recibido ningún entrenamiento.



### **Entrenamiento**


El siguiente código define la función de entrenamiento que se utiliza para entrenar el modelo.



In [ ]:
def train_model(modelo, optimizer, criterion, train_dataloader, valid_dataloader,  epochs=1000, save_dir="", file_name=None):
    cum_loss_list = []
    acc_epoch = []
    acc_old = 0
    model_path = os.path.join(save_dir, file_name)
    acc_dir = os.path.join(save_dir, os.path.splitext(file_name)[0] + "_acc")
    loss_dir = os.path.join(save_dir, os.path.splitext(file_name)[0] + "_loss")
    time_start = time.time()

    for epoch in tqdm(range(1, epochs + 1)):
        modelo.train()
        #print(modelo)
        #for parm in modelo.parameters():
        #    print(parm.requires_grad)
        
        cum_loss = 0
        for idx, (label, text) in enumerate(train_dataloader):
            optimizer.zero_grad()
            label, text = label.to(device), text.to(device)

            predicted_label = modelo(text)
            loss = criterion(predicted_label, label)
            loss.backward()
            #print(loss)
            torch.nn.utils.clip_grad_norm_(modelo.parameters(), 0.1)
            optimizer.step()
            cum_loss += loss.item()
        print(f"Epoca {epoch}/{epochs} - Pérdida: {cum_loss}")

        cum_loss_list.append(cum_loss)
        accu_val = evaluate_no_tqdm(valid_dataloader,modelo)
        acc_epoch.append(accu_val)

        if model_path and accu_val > acc_old:
            print(accu_val)
            acc_old = accu_val
            if save_dir is not None:
                pass
                #print("save model epoch",epoch)
                #torch.save(modelo.state_dict(), model_path)
                #save_list_to_file(lst=acc_epoch, filename=acc_dir)
                #save_list_to_file(lst=cum_loss_list, filename=loss_dir)

    time_end = time.time()
    print(f"Tiempo de entrenamiento: {time_end - time_start}")

#### **Entrenamiento en IMDB**

El siguiente código establece la tasa de aprendizaje (LR) en 1, que determina el tamaño del paso con el que el optimizador actualiza los parámetros del modelo durante el entrenamiento. El criterio `CrossEntropyLoss` se utiliza para calcular la pérdida entre las salidas predichas por el modelo y las etiquetas reales. Esta función de pérdida se emplea comúnmente en tareas de clasificación multiclase.

El optimizador elegido es `Stochastic Gradient Descent` (SGD), que ajusta los parámetros del modelo en función de los gradientes calculados respecto a la función de pérdida. El optimizador SGD utiliza la tasa de aprendizaje especificada para controlar el tamaño de las actualizaciones de los pesos.

Además, se define un programador (scheduler) de tasa de aprendizaje mediante `StepLR`. Este programador ajusta la tasa de aprendizaje durante el entrenamiento, reduciéndola en un factor (`gamma`) de 0.1 después de cada época (paso) para mejorar la convergencia y afinar el rendimiento del modelo. Estos componentes forman, en conjunto, la configuración esencial para entrenar una red neuronal usando la tasa de aprendizaje, el criterio de pérdida, el optimizador y el programador de tasa de aprendizaje especificados.

Por motivos de eficiencia de tiempo, **las siguientes líneas están comentadas y el modelo no se entrena realmente**. Si deseas echar un vistazo de cómo sería el proceso de entrenamiento, descomenta el siguiente bloque de código para entrenar el modelo durante 2 épocas. Si entrenaras este modelo en un escenario real, lo más probable es que aumentarías el número de épocas a una cifra mayor, como 100 o más. Dado el conjunto de entrenamiento reducido definido anteriormente, tarda aproximadamente 2 minutos completar 2 épocas de entrenamiento.

In [ ]:
'''
LR=1
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
save_dir = ""
file_name = "model_IMDB dataset small2.pth"
train_model(model=modelo, 
            optimizer=optimizer, 
            criterion=criterion, 
            train_dataloader=train_dataloader, 
            valid_dataloader=valid_dataloader, 
            epochs=2, 
            save_dir=save_dir, 
            file_name=file_name
           )
'''

Carguemos un modelo que ha sido preentrenado usando el mismo método pero con el conjunto de datos completo y durante 100 épocas.

El siguiente código traza el costo y la precisión de los datos de validación para cada época del modelo preentrenado hasta e incluyendo aquella época que obtuvo la mayor precisión. 

> Comprueba que el modelo preentrenado alcanzó una precisión de más del 85% en el conjunto de validación.

In [ ]:
acc_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/sybqacL5p1qeEO8d4xRZNg/model-IMDB%20dataset%20small2-acc')
loss_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/eOt6woGoaOB565T0RLH5WA/model-IMDB%20dataset%20small2-loss')
acc_epoch = pickle.load(acc_urlopened)
cum_loss_list = pickle.load(loss_urlopened)
plot(cum_loss_list,acc_epoch)

El siguiente código carga tu modelo preentrenado y evalúa su rendimiento en el conjunto de prueba. 

**Para mayor eficiencia, no ejecutaremos la evaluación porque puede tardar aproximadamente 4 minutos. En su lugar, muestra el resultado debajo de la celda. Si deseas confirmar el resultado por ti mismo, eres libre de descomentar la última línea en el siguiente bloque de código.**

In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/q66IH6a7lglkZ4haM6hB1w/model-IMDB%20dataset%20small2.pth')
model_ = Net(vocab_size=vocab_size, num_class=2).to(device)
model_.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))
#evaluate(test_dataloader, model_)

> Comprueba que el modelo preentrenado alcanzó una exactitud de aproximadamente el 83% en los datos de prueba.


#### **Ajuste fino de un modelo preentrenado en el conjunto de datos AG News**

En lugar de entrenar un modelo en el conjunto de datos IMDB como hicimos anteriormente, puedes ajustar finamente (fine-tune) un modelo que ha sido preentrenado en el conjunto de datos AG News, que es una colección de artículos de noticias.  El objetivo del conjunto AG News es categorizar los artículos en una de cuatro categorías: deportes, negocios, ciencia/tecnología o mundo. 

Comenzaremos entrenando un modelo desde cero en el conjunto AG News. Para ahorrar tiempo, puedes hacerlo en una sola celda. 

Además, por eficiencia, **comenta la parte de entrenamiento**. Si deseas entrenar el modelo durante 2 épocas en un conjunto de datos reducido para demostrar cómo sería el proceso de entrenamiento, descomenta la sección que dice `### Descomenta para entrenar ###` antes de ejecutar la celda. 

Entrenar durante 2 épocas en el conjunto reducido puede llevar aproximadamente 3 minutos.


In [ ]:
# Carga los datos de entrenamiento de AG News
train_iter_ag_news = AG_NEWS(split="train")

# Calcula el número de clases distintas en AG News
num_class_ag_news = len(set([label for (label, text) in train_iter_ag_news ]))
num_class_ag_news

# Divide el conjunto de datos en iteradores de entrenamiento y prueba
train_iter_ag_news, test_iter_ag_news = AG_NEWS()

# Convierte los iteradores de entrenamiento y prueba en datasets de tipo mapa
train_dataset_ag_news = to_map_style_dataset(train_iter_ag_news)
test_dataset_ag_news = to_map_style_dataset(test_iter_ag_news)

# Determina el número de muestras para entrenamiento y validación (5% para validación)
num_train_ag_news = int(len(train_dataset_ag_news) * 0.95)

# Divide aleatoriamente el dataset de entrenamiento en entrenamiento y validación
# El 95% de las muestras irán a entrenamiento y el 5% restante a validación
split_train_ag_news_, split_valid_ag_news_ = random_split(
    train_dataset_ag_news,
    [num_train_ag_news, len(train_dataset_ag_news) - num_train_ag_news]
)

# Reduce el conjunto de entrenamiento para que se ejecute rápidamente como ejemplo
# SI DESEAS ENTRENAR EN EL DATASET AG_NEWS, COMENTA LAS 2 LÍNEAS DE ABAJO.
# SIN EMBARGO, TEN EN CUENTA QUE EL ENTRENAMIENTO TOMARÁ MUCHO TIEMPO
num_train_ag_news = int(len(train_dataset_ag_news) * 0.05)
split_train_ag_news_, _ = random_split(
    split_train_ag_news_,
    [num_train_ag_news, len(split_train_ag_news_) - num_train_ag_news]
)

# Define el dispositivo: usa GPU si está disponible, de lo contrario CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Función para convertir la etiqueta (1–4) a un índice en 0–3
def label_pipeline(x):
    return int(x) - 1

from torch.nn.utils.rnn import pad_sequence

# Función collate para procesar un lote: convierte etiquetas y secuencias a tensores y aplica padding
def collate_batch_ag_news(batch):
    label_list, text_list = [], []
    for _label, _text in batch:
        # Convierte y almacena cada etiqueta
        label_list.append(label_pipeline(_label))
        # Tokeniza y almacena cada texto como tensor de enteros
        text_list.append(torch.tensor(text_pipeline(_text), dtype=torch.int64))

    # Crea tensor de etiquetas
    label_list = torch.tensor(label_list, dtype=torch.int64)
    # Aplica padding para igualar la longitud de las secuencias
    text_list = pad_sequence(text_list, batch_first=True)

    # Devuelve los tensores en el dispositivo correspondiente
    return label_list.to(device), text_list.to(device)

# Tamaño de lote
BATCH_SIZE = 32

# Crea DataLoaders para entrenamiento, validación y prueba
train_dataloader_ag_news = DataLoader(
    split_train_ag_news_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch_ag_news
)
valid_dataloader_ag_news = DataLoader(
    split_valid_ag_news_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch_ag_news
)
test_dataloader_ag_news = DataLoader(
    test_dataset_ag_news, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch_ag_news
)

# Crea la instancia del modelo y muévela al dispositivo
model_ag_news = Net(num_class=4, vocab_size=vocab_size).to(device)
model_ag_news.to(device)

'''
### Descomenta para entrenar ###
LR = 1
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_ag_news.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
save_dir = ""
file_name = "model_AG News small1.pth"
train_model(
    model=model_ag_news,
    optimizer=optimizer,
    criterion=criterion,
    train_dataloader=train_dataloader_ag_news,
    valid_dataloader=valid_dataloader_ag_news,
    epochs=2,
    save_dir=save_dir,
    file_name=file_name
)
'''


Carguemos un modelo que ha sido preentrenado usando el mismo método pero con el conjunto completo de AG News durante 100 épocas.

El siguiente código traza el costo y la precisión de los datos de validación para cada época del modelo preentrenado hasta e incluyendo la época que obtuvo la mayor precisión. 

> Puedes verificar que el modelo preentrenado alcanzó una precisión muy alta de más del 90% en el conjunto de validación de AG News.


In [ ]:
acc_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bQk8mJu3Uct3I4JEsEtRnw/model-AG%20News%20small1-acc')
loss_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/KNQkqJWWwY_XfbFBRFhZNA/model-AG%20News%20small1-loss')
acc_epoch = pickle.load(acc_urlopened)
cum_loss_list = pickle.load(loss_urlopened)
plot(cum_loss_list,acc_epoch)

El siguiente código carga el modelo preentrenado y evalúa su rendimiento en el conjunto de prueba de AG News. 

**Por eficiencia, no ejecutemos la evaluación porque puede tardar unos minutos. En su lugar, indica que el modelo preentrenado funciona bien en el conjunto de AG News. Si deseas confirmar el resultado, siéntete libre de descomentar la última línea en el siguiente bloque de código.**



In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/9c3Dh2O_jsYBShBuchUNlg/model-AG%20News%20small1.pth')
model_ag_news_ = Net(vocab_size=vocab_size, num_class=4).to(device)
model_ag_news_.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))
#evaluate(test_dataloader_ag_news, model_ag_news_)

> ¿El modelo preentrenado funcionó extremadamente bien en el conjunto de datos AG News?, ¿se puede ajustar este modelo para que también funcione bien en el conjunto IMDB?.




In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/9c3Dh2O_jsYBShBuchUNlg/model-AG%20News%20small1.pth')
model_fine1 = Net(vocab_size=vocab_size, num_class=4).to(device)
model_fine1.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))


El conjunto de datos IMDB es una tarea de clasificación binaria con solo dos clases (reseñas positivas y negativas). Por lo tanto, la capa de salida del modelo AG News debe ajustarse para tener solo dos neuronas de salida, a fin de reflejar la naturaleza binaria del conjunto IMDB. 

Este ajuste es esencial para que el modelo aprenda y prediga con precisión el sentimiento de las reseñas de películas en el conjunto de datos IMDB.


In [ ]:
model_fine1.classifier
in_features = model_fine1.classifier.in_features
print("Capa final original:", model_fine1.classifier)
print("Dimensión de entrada de la capa final:", in_features)

Reemplaza la capa final para resolver un problema de dos clases.

In [ ]:
model_fine1.classifier = nn.Linear(in_features, 2)
model_fine1.to(device)

El siguiente código muestra las capas que están congeladas (`requires_grad == False`) y descongeladas (`requires_grad == True`) en el modelo. 

Las capas descongeladas tendrán sus pesos actualizados durante el ajuste fino.


In [ ]:
for name, param in model_fine1.named_parameters():
    print(f"{name} requires_grad: {param.requires_grad}")

El siguiente bloque de código simula el ajuste fino en el conjunto de entrenamiento reducido durante solo 2 épocas. 

**Por motivos de eficiencia de tiempo, este bloque de código está comentado**. Si quieres ver cómo sería el entrenamiento, descomenta el siguiente bloque de código, pero recuerda que este proceso podría tardar aproximadamente 2 minutos en ejecutarse.


In [ ]:
'''
LR=1
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_fine1.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
save_dir = ""
file_name = "model_fine1.pth"
train_model(model=model_fine1, optimizer=optimizer, criterion=criterion, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader,  epochs=2,  save_dir=save_dir ,file_name=file_name )
'''

El siguiente código muestra el progreso del ajuste fino completo de todo el conjunto de entrenamiento de IMDB durante 100 épocas.


In [ ]:
acc_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/3LEJw8BRgJJFGqlLxaETxA/model-fine1-acc')
loss_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-CT1h97vjv0TolY82Nw29g/model-fine1-loss')
acc_epoch = pickle.load(acc_urlopened)
cum_loss_list = pickle.load(loss_urlopened)
plot(cum_loss_list,acc_epoch)

La siguiente línea carga un modelo preajustado que fue entrenado durante 100 épocas con el conjunto completo de IMDB y evalúa su rendimiento en el conjunto de prueba de IMDB. 

**Por motivos de eficiencia, no ejecutemos la evaluación ya que puede tardar varios minutos. En su lugar, muestra el resultado debajo de la celda. Si deseas comprobar el resultado por ti mismo, siéntete libre de descomentar la última línea del bloque de código.**



In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/e0WOHKh5dnrbC2lGhpsMMw/model-fine1.pth')
model_fine1_ = Net(vocab_size=vocab_size, num_class=2).to(device)
model_fine1_.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))
#evaluate(test_dataloader, model_fine1_)

Este modelo debe demostrar una mejora notable, logrando un rendimiento excepcional con una exactitud del 86 % en los datos de prueba. Esto es superior al 83 % conseguido por el modelo entrenado desde cero con el conjunto de datos IMDB. Aunque el proceso de entrenamiento fue intensivo en tiempo (el ajuste fino fue tan costoso en tiempo como entrenar el modelo desde cero), el rendimiento mejorado subraya la eficacia y superioridad del modelo ajustado frente al modelo entrenado desde cero. 

Gran parte del esfuerzo computacional se dedicó a actualizar las capas del transformer. Para acelerar el proceso de entrenamiento, una estrategia viable es centrarse únicamente en entrenar la capa final, lo que puede reducir significativamente la carga computacional, aunque podría comprometer la precisión del modelo.



#### **Ajuste fino solo de la capa final**

El ajuste fino de la capa de salida final de una red neuronal es similar al ajuste fino de todo el modelo. Puedes comenzar cargando el modelo preentrenado que deseas afinar. 

En este caso, se trata del mismo modelo preentrenado en el conjunto de datos AG News.



In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/9c3Dh2O_jsYBShBuchUNlg/model-AG%20News%20small1.pth')
model_fine2 = Net(vocab_size=vocab_size, num_class=4).to(device)
model_fine2.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))

Ahora, la diferencia clave. Iteras a través de todos los parámetros del modelo `model_fine2` y estableces el atributo `requires_grad` de cada parámetro en `False`. Esto congela efectivamente todas las capas del modelo, lo que significa que sus pesos serán actualizados durante el entrenamiento.


In [ ]:
# Congela todas las capas del modelo
for param in model_fine2.parameters():
    param.requires_grad = False

Reemplaza la capa final para reflejar que estás resolviendo un problema de dos clases. Observa que la nueva capa no estará congelada.

In [ ]:
dim=model_fine2.classifier.in_features

In [ ]:
model_fine2.classifier = nn.Linear(dim, 2)

In [ ]:
model_fine2.to(device)


El siguiente bloque simula el fine-tuning en el conjunto de entrenamiento reducido durante solo 2 épocas. **Para ahorrar tiempo, este bloque de código ha sido comentado**. 

El código debería tardar menos en entrenar que el fine-tuning completo realizado anteriormente, ya que solo la capa final no está congelada.



In [ ]:
'''
LR=1
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_fine2.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
save_dir = ""
file_name = "model_fine2.pth"
train_model(model=model_fine2, optimizer=optimizer, criterion=criterion, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader,  epochs=2,  save_dir=save_dir ,file_name=file_name )
'''

Una vez más, no usarás el modelo que acabas de ajustar, sino que inspeccionarás el proceso de fine-tuning de la capa final de un modelo ajustado en el conjunto completo de entrenamiento de IMDB durante 100 épocas.

In [ ]:
acc_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/UdR3ApQnxSeV2mrA0CbiLg/model-fine2-acc')
loss_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/rWGDIF-uL2dEngWcIo9teQ/model-fine2-loss')
acc_epoch = pickle.load(acc_urlopened)
cum_loss_list = pickle.load(loss_urlopened)
plot(cum_loss_list,acc_epoch)

La siguiente línea carga el modelo preentrenado y evalúa su rendimiento en el conjunto de prueba. 

**Para mayor eficiencia, no ejecutaremos la evaluación porque puede tardar algunos minutos. En su lugar, informa el resultado debajo de la celda. Si deseas confirmar el resultado por ti mismo, siéntete libre de descomentar la última línea del siguiente bloque de código.**

In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/B-1H6lpDg-A0zRwpB6Ek2g/model-fine2.pth')
model_fine2_ = Net(vocab_size=vocab_size, num_class=2).to(device)
model_fine2_.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))
#evaluate(test_dataloader, model_fine2_)

El código anterior indica que, aunque el fine-tuning de la capa final tarda significativamente menos tiempo que el fine-tuning de todo el modelo, el rendimiento del modelo con solo la última capa sin congelar es mucho peor (64 % de exactitud) que el modelo ajustado con todas las capas sin congelar (86 % de exactitud).


### **Adaptadores**

**FeatureAdapter** es un módulo de red neuronal que introduce un cuello de botella de baja dimensión en una arquitectura Transformer para permitir fine-tuning con menos parámetros. Comprime los embeddings de alta dimensión originales a una dimensión menor, aplica una transformación no lineal y luego las expande de nuevo a la dimensión original. 

Este proceso va seguido de una conexión residual que añade la salida transformada a la entrada original para preservar la información y promover el flujo de gradiente.

**Beneficios de usar adaptadores en redes neuronales**

* **Fine-tuning eficiente**: Los adaptadores permiten actualizaciones focalizadas en partes específicas del modelo, reduciendo la necesidad de reentrenar grandes secciones de la red.
* **Eficiencia de parámetros**: Al agregar solo unos pocos parámetros, los adaptadores hacen factible modificar modelos grandes sin una carga computacional sustancial.
* **Preservación de las características preentrenadas**: Los adaptadores permiten la modificación de un modelo mientras se retienen las valiosas características aprendidas durante el preentrenamiento extenso.
* **Modularidad y flexibilidad**: Mejoran la modularidad de los modelos, permitiendo una fácil adaptación a diversas tareas sin alterar la arquitectura central.
* **Adaptación específica para la tarea**: Los adaptadores pueden diseñarse para mejorar el rendimiento en tareas particulares, optimizando la efectividad del modelo.
* **Transferencia de aprendizaje y adaptación de dominio**: Facilitan la adaptación de modelos a nuevos dominios, cerrando brechas entre diferentes distribuciones de datos.
* **Aprendizaje continuo**: Los adaptadores apoyan la capacidad del modelo para aprender nueva información continuamente sin olvidar conocimientos previos.
* **Reducción del riesgo de sobreajuste**: Con menos parámetros entrenables, los adaptadores ayudan a prevenir el sobreajuste, especialmente en conjuntos de datos reducidos.

El siguiente código muestra un modelo de adaptador:

In [ ]:
class FeatureAdapter(nn.Module):
    """
    Atributos:
        size (int): La dimensión del bottleneck a la cual se reducen temporalmente los embeddings.
        model_dim (int): La dimensión original de los embeddings o características en el modelo Transformer.
    """
    def __init__(self, bottleneck_size=50, model_dim=100):
        super().__init__()
        self.bottleneck_transform = nn.Sequential(
            nn.Linear(model_dim, bottleneck_size),  # Proyectar a una dimensión más pequeña
            nn.ReLU(),                              # Aplicar no linealidad
            nn.Linear(bottleneck_size, model_dim)   # Proyectar de nuevo a la dimensión original
        )

    def forward(self, x):
        """
        Paso hacia adelante (forward) de FeatureAdapter. Aplica la transformación del bottleneck al tensor
        de entrada y añade una conexión residual.

        Args:
            x (Tensor): Tensor de entrada con forma (batch_size, seq_length, model_dim).

        Returns:
            Tensor: Tensor de salida después de aplicar la transformación del adaptador y la conexión residual,
                    manteniendo la forma original de la entrada.
        """
        transformed_features = self.bottleneck_transform(x)  # Transforma características a través del embudo
        output_with_residual = transformed_features + x      # Añade la conexión residual
        return output_with_residual

La clase `adapted` envuelve esta funcionalidad de adaptador alrededor de cualquier capa lineal especificada, mejorando su salida con la no linealidad de una función de activación ReLU. 

Esta configuración es especialmente útil para experimentar con modificaciones arquitectónicas sutiles en modelos de deep learning, facilitando el fine-tuning y potencialmente mejorando el rendimiento del modelo en tareas complejas.



In [ ]:
class Adapted(nn.Module):
    def __init__(self, linear, bottleneck_size=None):
        super(Adapted, self).__init__()
        self.linear = linear
        model_dim = linear.out_features
        if bottleneck_size is None:
            bottleneck_size = model_dim // 2   # Define el tamaño del cuello de botella por defecto como la mitad de model_dim

        # Inicializa FeatureAdapter con el bottleneck_size y model_dim calculados
        self.adaptor = FeatureAdapter(bottleneck_size=bottleneck_size, model_dim=model_dim)

    def forward(self, x):
        # Primero, la entrada x pasa por la capa lineal
        x = self.linear(x)
        # Luego se adapta usando FeatureAdapter
        x = self.adaptor(x)
        return x

A continuación, cargamos el modelo transformer preentrenado que fue entrenado sobre el dataset AG News:


In [ ]:
urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/9c3Dh2O_jsYBShBuchUNlg/model-AG%20News%20small1.pth')
model_adapters = Net(vocab_size=vocab_size, num_class=4).to(device)
model_adapters.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))

Luego , se congela los parámetros del modelo llamado `model_adapters` para evitar que se actualicen durante el entrenamiento. Luego, obtienes el número de características de entrada del clasificador y reemplazas dicho clasificador por una nueva capa lineal que produce dos clases:


In [ ]:
for param in model_adapters.parameters():
    param.requires_grad = False

dim= model_adapters.classifier.in_features
model_adapters.classifier = nn.Linear(dim, 2)

Veamos cómo aplicar el objeto `adaptado` a una capa lineal para obtener la primera salida. Primero, obtienes la capa lineal original:

In [ ]:
mi_example_layer=model_adapters.transformer_encoder.layers[0].linear1
print(mi_example_layer)

En el siguiente código, se copia la capa lineal y se añade una capa adaptadora:


In [ ]:
mi_adapeted_layer=Adapted(mi_example_layer)
print(mi_adapeted_layer)

Puedes imprimir la capa adaptada y mostrar que los nuevos parámetros tienen su atributo `requires_grad` en `True`, indicando que se actualizarán durante el entrenamiento:


In [ ]:
for parm in mi_adapeted_layer.parameters():
    print(parm.requires_grad)

Podrías asignar directamente esta capa adaptada en el modelo, pero como hay muchas capas, es más sistemático recorrerlas y reemplazar aquellas que te interesan. Ten en cuenta que al fijar el tamaño del cuello de botella en 24, habrá menos parámetros que entrenar comparado con el fine-tuning completo:


In [ ]:
# Adapta una capa específica (comentado)
#model_adapters.transformer_encoder.layers[0].linear1=Adapted(mi_example_layer)

In [ ]:
#  Encuentra el número de capas
N_layers=len(model_adapters.transformer_encoder.layers)

In [ ]:
# Recorre el modelo y adaptar
for n in range(N_layers):
    encoder = model_adapters.transformer_encoder.layers[n]
    if encoder.linear1:
        print(" antes de linear1")
        print(encoder.linear1)
        model_adapters.transformer_encoder.layers[n].linear1 = Adapted(encoder.linear1, bottleneck_size=24)
        print(" después de linear1")
        print(model_adapters.transformer_encoder.layers[n].linear1)

    if encoder.linear2:
        print(" antes de linear2")
        print(encoder.linear2)
        model_adapters.transformer_encoder.layers[n].linear2 = Adapted(encoder.linear2, bottleneck_size=24)
        print(" después de linear2")
        print(model_adapters.transformer_encoder.layers[n].linear2)

Envía el modelo al dispositivo:


In [ ]:
# Envia modelo al dispositivo
model_adapters.to(device)

Finalmente, simulamos el entrenamiento del modelo adaptado entrenándolo en un subconjunto reducido de IMDB durante 2 épocas:


In [ ]:
LR=1
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_adapters.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
save_dir = ""
file_name = "model_adapters.pth"
train_model(modelo=model_adapters, optimizer=optimizer, criterion=criterion, train_dataloader=train_dataloader, valid_dataloader=valid_dataloader,  epochs=2,  save_dir=save_dir ,file_name=file_name )


Naturalmente, no usaremos este modelo entrenado "a mano". En su lugar, seguiremos el entrenamiento de un modelo adaptado fine-tuneado sobre el conjunto completo de IMDB durante 100 épocas:


In [ ]:
acc_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/D49zrrMPWO_ktwQo7PSHIQ/model-adapters-acc')
loss_urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/RXWlmyaco695RiaoU7QsnA/model-adapters-loss')
acc_epoch = pickle.load(acc_urlopened)
cum_loss_list = pickle.load(loss_urlopened)
plot(cum_loss_list,acc_epoch)

El siguiente bloque de código carga el modelo adaptado fine-tuneado durante 100 épocas en el conjunto completo de IMDB y evalúa su rendimiento sobre el conjunto de prueba:


In [ ]:
model_adapters_ = Net(vocab_size=vocab_size, num_class=2).to(device)
for n in range(N_layers):
    encoder = model_adapters_.transformer_encoder.layers[n]
    if encoder.linear1:
        print(" antes de linear1")
        print(encoder.linear1)
        model_adapters_.transformer_encoder.layers[n].linear1 = Adapted(encoder.linear1, bottleneck_size=24)
        print(" después de linear1")
        print(model_adapters_.transformer_encoder.layers[n].linear1)

    if encoder.linear2:
        print(" antes de linear2")
        print(encoder.linear2)
        model_adapters_.transformer_encoder.layers[n].linear2 = Adapted(encoder.linear2, bottleneck_size=24)
        print(" después de linear2")
        print(model_adapters_.transformer_encoder.layers[n].linear2)

model_adapters_.to(device)
for param in model_adapters_.parameters():
    param.requires_grad = False  # Congela todos los parámetros para la evaluación

urlopened = urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/PGhd5G_NVrWNH-_jdjwNlw/model-adapters.pth')
model_adapters_.load_state_dict(torch.load(io.BytesIO(urlopened.read()), map_location=device))
evaluate(test_dataloader, model_adapters_)  # Evalúa en el conjunto de prueba

Como puedes observar, el rendimiento del modelo adaptado fine-tuneado es prácticamente idéntico al del modelo completamente fine-tuneado, alcanzando ambos alrededor de un 86% de exactitud. Esto resulta especialmente sorprendente, pues se actualizaron muchos menos pesos en el modelo adaptado que en el modelo completo. Ten en cuenta que solo las capas de adaptadores con un bottleneck de 24 y la capa final del clasificador están descongeladas.




Lo anterior demuestra que los adaptadores pueden usarse para un fine-tuning eficiente en parámetros (PEFT) y que el rendimiento de un modelo fine-tuneado mediante adaptadores puede ser casi tan bueno como el de un modelo fine-tuneado completamente con todas las capas descongeladas.

#### **Ejercicio: Adaptar capas lineales en una red diferente**

El siguiente código define una red neuronal llamada `NeuralNetwork`:


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

exercise_model = NeuralNetwork()

exercise_model.to(device)
for param in exercise_model.parameters():
    param.requires_grad = False

print(exercise_model)

`NeuralNetwork` es una red neuronal que usa el contenedor `Sequential` de PyTorch. Adapta las dos primeras capas lineales del contenedor `Sequential` usando el adaptador de cuello de botella con un tamaño de cuello de 30. Además, cambia la última capa lineal por una capa que produzca 5 salidasT


In [ ]:
#Tu respuesta